In [2]:

from pyspark.sql.functions import col, coalesce, lit,to_date
from pyspark.sql.types import IntegerType, StringType, DateType


StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 4, Finished, Available, Finished)

In [3]:
%run SLV_NB_Functions_Files

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 5, Finished, Available, Finished)

In [4]:
tickets_df = get_latest_file(
    lakehouse_path=f"abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Files/User Created Data/Tickets",
    # src_table="tickets"
)

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 6, Finished, Available, Finished)

Latest file for abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Files/User Created Data/Tickets: abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Files/User Created Data/Tickets/2025-05-02Vtas_x2.csv


In [5]:
tickets_df.createTempView('temp_tickets_csv')

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 7, Finished, Available, Finished)

In [10]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW temp_fact_tickets AS
WITH sales_agg AS (
    SELECT  
        CAST(s.tclave AS INT) AS store_id,
        s.fecha,
        COUNT(DISTINCT s.PK_TICKET) AS ticket_count,
        SUM(s.atmcant) AS qty_per_ticket
    FROM sales_items_golden s
    JOIN dimproducts p 
        ON s.ICLAVE = p.src_product_id
    --WHERE CAST(s.fecha AS DATE) >= DATEADD(DAY, -3, CURRENT_DATE)
    GROUP BY s.tclave, s.fecha
),
filtered_sales AS (
    SELECT 
        store_id_from, 
        movement_date,
        SUM(net_sales) as net_sales
    FROM fact_sales fs
    JOIN dimproducts dp 
        ON fs.product_id = dp.src_product_id
    --WHERE CAST(fs.movement_date AS DATE) >= DATEADD(DAY, -3, CURRENT_DATE)
    GROUP BY store_id_from, movement_date
)
SELECT 
    sa.store_id,
    sa.fecha,
    sa.ticket_count,
    sa.qty_per_ticket,
    fs.net_sales
FROM sales_agg sa
JOIN filtered_sales fs
    ON fs.store_id_from = sa.store_id
   AND fs.movement_date = sa.fecha;

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
%%sql 
select * from temp_fact_tickets
where store_id = 64
and fecha = '2025-01-02'

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 15, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 5 fields>

In [7]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW temp_fact_ticket AS

WITH 
-- Datos de 2025 de temp_fact_tickets
fact_2025 AS (
  SELECT 
    store_id,
    fecha, 
    ticket_count,
    qty_per_ticket,
    net_sales
  FROM temp_fact_tickets
  WHERE fecha > '2024-12-31'
),

-- Datos de 2025 de temp_tickets_csv
csv_2025 AS (
  SELECT 
    Tclave AS store_id,
    date_format(to_date(dsfecha, 'dd/MM/yyyy'), 'yyyy-MM-dd') AS fecha,
    trans AS ticket_count,
    0 AS qty_per_ticket,
    0 AS net_sales
  FROM temp_tickets_csv
  WHERE to_date(dsfecha, 'dd/MM/yyyy') > '2024-12-31'
),

-- Combinación de 2025
combined_2025 AS (
  SELECT * FROM fact_2025
  UNION ALL
  SELECT * FROM csv_2025
),

-- Datos históricos (2024 y anteriores)
historical_data AS (
  SELECT 
    store_id,
    fecha, 
    ticket_count,
    qty_per_ticket,
    net_sales
  FROM temp_fact_tickets
  WHERE fecha <= '2024-12-31'
)

-- Unión final
SELECT 
  store_id,
  fecha,
  SUM(ticket_count) AS ticket_count,
  SUM(qty_per_ticket) AS qty_per_ticket,
  SUM(net_sales) AS net_sales
FROM combined_2025
GROUP BY store_id, fecha

UNION ALL

SELECT * FROM historical_data

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
%%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_fact_ticket AS

WITH 
-- Datos históricos (2024 y anteriores)
historical_data AS (
  SELECT 
    store_id,
    fecha, 
    ticket_count,
    qty_per_ticket,
    net_sales
  FROM temp_fact_tickets
  WHERE fecha <= '2024-12-31'
),

-- Datos de 2025 combinando ambas fuentes
current_year_data AS (
  SELECT 
    COALESCE(t1.store_id, t2.store_id) AS store_id,
    COALESCE(t1.fecha, t2.fecha) AS fecha,
    COALESCE(t1.ticket_count, 0) + COALESCE(t2.trans, 0) AS ticket_count,
    COALESCE(t1.qty_per_ticket, 0) AS qty_per_ticket,
    COALESCE(t1.net_sales, 0) AS net_sales
  FROM 
    (SELECT * FROM temp_fact_tickets WHERE fecha > '2024-12-31') t1
    FULL OUTER JOIN
    (SELECT 
       Tclave AS store_id,
       date_format(to_date(dsfecha, 'dd/MM/yyyy'), 'yyyy-MM-dd') AS fecha,
       trans
     FROM temp_tickets_csv
     WHERE to_date(dsfecha, 'dd/MM/yyyy') > '2024-12-31') t2
    ON t1.store_id = t2.store_id AND t1.fecha = t2.fecha
)

-- Union de los Datos
SELECT 
  store_id,
  fecha,
  ticket_count,
  qty_per_ticket,
  net_sales
FROM historical_data

UNION ALL

SELECT 
  store_id,
  fecha,
  ticket_count,
  qty_per_ticket,
  net_sales
FROM current_year_data

StatementMeta(, e5451d80-fbf1-41f6-acf9-dd8c1cd5c0a7, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
%%sql
delete from fact_tickets

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [9]:
%%sql
MERGE INTO fact_tickets AS target
USING temp_fact_ticket AS source
ON target.store_id = source.store_id AND target.fecha = source.fecha

WHEN MATCHED THEN
    UPDATE SET 
        target.ticket_count = source.ticket_count,
        target.qty_per_ticket = source.qty_per_ticket,
        target.net_sales = source.net_sales

WHEN NOT MATCHED THEN
    INSERT (store_id, fecha, ticket_count, qty_per_ticket, net_sales)
    VALUES (source.store_id, source.fecha, source.ticket_count, source.qty_per_ticket, source.net_sales);

StatementMeta(, e7090b57-e751-41d3-ad92-2560f86df4b0, 11, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>